<p style="font-size:32px; font-weight: bolder; text-align: center"> Colored-noise methods </p>

This notebook provides a hands-on counterpart to the "Colored-noise methods" lecture for the MOOC "Path Integrals in Atomistic Modeling". If you haven't done so already, check the [getting started](0-getting_started.ipynb) notebook to make sure that the software infrastructure is up and running. 

The different sections in this notebook match the parts this lecture is divided into:

1. [Generalized Langevin Equations](#gle)
2. [Equilibrium GLE sampling](#equilibrium)
3. [Non-Equilibrium GLE sanpling](#non-equilibrium)
4. [Combining GLE and PIMD](#pi-gle)
5. [Dynamical properties](#dynamics)

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import ase, ase.io
from ase.ga.utilities import get_rdf
import chemiscope
import pimdmooc
pimdmooc.add_ipi_paths()

<a id="gle"> </a>

# Generalized Langevin Equations 

This section provides a brief demonstration of the possibility of computing properties of a generalized Langevin equation written in the extended-phase-space form, as a function of the drift matrix $\mathbf{A}_p$ and the diffusion matrix $\mathbf{B}_p$ (the latter being usually fixed by a fluctuation-dissipation relation.

A matrix $\mathbf{M}_p$ coupling physical momentum $p$ and extended momenta $\mathbf{s}$ can be written down as a combination of blocks, following the convention

$$
\begin{array}{c|cc}
      &    p   &   \mathbf{s} \\ \hline
   p  & m_{pp} &  \mathbf{m}_p^T \\
\mathbf{s} & \bar{\mathbf{m}}_p &  \mathbf{M}\\
\end{array}
$$

In [ ]:
# this splits up a Mp matrix into the four blocks describing interactions between p and s
def gle_split(Mp):
    """ Splits a matrix in the various blocks """
    return Mp[:1,:1], Mp[:1, 1:], Mp[1:, :1], Mp[1:,1:]

The simplest property that can be computed from the GLE matrices are the memory kernels of the associated non-Markovian formulation. The friction kernel reads

$$
K(t) = a_{pp} \delta(t) - \mathbf{a}_p^T e^{-t \mathbf{A}} \bar{\mathbf{a}}_p
$$

and its Fourier transform 

$$
K(\omega) = 2 a_{pp} -2 \mathbf{a}_p^T \frac{\mathbf{A}}{\mathbf{A}^2+\omega^2} \bar{\mathbf{a}}_p
$$

In [ ]:
def Kt(Ap, t, with_delta=False):
    app, ap, bap, A = gle_split(Ap)
    return (app[0,0] if with_delta and t==0 else 0) - (ap@sp.linalg.expm(-A*t)@bap)[0,0]

def Kw(Ap, w):
    app, ap, bap, A = gle_split(Ap)
    return (2*app  - 2*(ap@A)@np.linalg.solve(A@A+np.eye(len(A))*w**2,bap))[0,0]

## Memory kernels

Different analytical forms of memory kernel can be obtained with appropriate parameterizations of the drift matrix. The ones given below yield $K(t)$ that are exponential, $K(t)=\gamma e^{-t/\tau}$ or different types of Dirac-$\delta$-like functions, that give a peaked memory kernel in the frequency domain (the corresponding functional form is rather cumbersome, you can find a thorough discussion [here](https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/152344/eth-2145-02.pdf)) - the functional form has been slightly modified to give more intuitive link between the parameters and the shape of $K(\omega)$. In all these cases, the parameter `app`, corresponding to $a_{pp}$, introduces a non-Markovian term in the kernel.

In [ ]:
def Ap_exp(gamma, tau, app=0):
    """ Drift matrix for an exponential memory kernel.
    gamma: intensity of the friction
    tau: time scale of the exponential decay
    app: Markovian term
    """
    return np.asarray( [ [app,-np.sqrt(gamma/tau) ], [np.sqrt(gamma/tau), 1/tau ]])
def Ap_delta(gamma, omega0, domega, app=0):
    """ Drift matrix for a delta-like memory kernel.  
    gamma: intensity of the friction
    omega0: frequency-domain center of the K(w) peak
    domega: width of the peak
    app: Markovian term
    """
    return np.asarray( [ [app, np.sqrt(gamma*domega/2), np.sqrt(gamma*domega/2) ], 
                         [-np.sqrt(gamma*domega/2),  domega, omega0 ],
                         [-np.sqrt(gamma*domega/2), -omega0, domega ]
                       ])
def Ap_delta_alt(gamma, omega0, domega, app=0):
    """ Drift matrix for a delta-like memory kernel. Alternative form with K(0)=0. 
    gamma: intensity of the friction
    omega0: frequency-domain center of the K(w) peak
    domega: width of the peak
    app: Markovian term
    """
    return np.asarray( [ [app,np.sqrt(gamma*domega/2), 0 ], 
                         [-np.sqrt(gamma*domega/2),  domega, omega0 ],
                         [0, -omega0, 0 ]
                       ])

Below you can plot $K(\omega)$ for the three functional forms above. Play around with the parameters to verify their effect on the shape of the memory kernel spectrum. 

In [ ]:
wgrid = np.geomspace(1e-3,1e3,200)
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
# defaults: Ap_delta_alt(1, 1, 1, 1e-8)
ax.loglog(wgrid, [Kw(Ap_delta_alt(1, 1, 0.1, 1e-8), w) for w in wgrid], 'r-' )  
# defaults: Ap_delta(1, 0.1, 0.01, 1e-8)
ax.loglog(wgrid, [Kw(Ap_delta(1, 0.1, 0.01, 1e-8), w) for w in wgrid], 'b-' ) 
# defaults: Ap_exp(1, 1, 0)
ax.loglog(wgrid, [Kw(Ap_exp(1, 1, 0), w) for w in wgrid], 'k-' )
ax.set_xlabel(r"$\omega$ / a.u."); ax.set_ylabel(r"$K(\omega)$ / a.u."); 

An important idea that makes it easy to reuse GLE parameters for different systems is that it is possible to "translate" the shape of $K(\omega)$ by scaling it by a factor $\alpha$. This is essentially a change of units, so scaling the kernel moves the curve right and up in the $(\omega, K)$ plane

In [ ]:
wgrid = np.geomspace(1e-3,1e3,200)
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.loglog(wgrid, [Kw(Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r-' )
ax.loglog(wgrid, [Kw(0.1*Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r:' )
ax.loglog(wgrid, [Kw(10*Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r--' )
ax.set_xlabel(r"$\omega$ / a.u."); ax.set_ylabel(r"$K(\omega)$ / a.u."); 

<p style="color:blue; font-weight:bold">
In the case of the analytical memory functions above, you can actually also mimic the effect of scaling by changing the value of the parameters. What parameters should you use for Ap_delta below, so that the blue curve becomes identical to the red curve?
</p>

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.loglog(wgrid, [Kw(0.1*Ap_delta(1,1,0.1,1e-4), w) for w in wgrid], 'r--' )

# modify the parameters below ↓ ↓ ↓ ↓, corresponding to gamma, omega0, domega, app
ax.loglog(wgrid, [Kw(Ap_delta(1,1,1,0), w) for w in wgrid], 'b:' )

## GLE integration

We now see how a GLE can be integrated in its Markovian, extended-phase-space form. The idea is very similar to the integrator for Langevin dynamics: a free-particle propagator that propagates $(p, \mathbf{s})$ for a finite time step $dt$ without an external potential is combined with a velocity-Verlet integrator for the Hamiltonian part of the equations of motion. The GLE integrator can be formulated as 

$$
(p, \mathbf{s})^T \leftarrow \mathbf{T}_p (p, \mathbf{s})^T + \mathbf{S}_p \boldsymbol{\xi}^T
$$

where $\boldsymbol{\xi}$ is a vector of uncorrelated random numbers, $\mathbf{T}_p = e^{-\mathbf{A}_p dt}$, and $\mathbf{S}_p\mathbf{S}_p^T = \mathbf{C}_p - \mathbf{T}_p \mathbf{C}_p \mathbf{T}_p^T$. 

In [ ]:
# Example classes for VV and GLE integration. Should be rather self-explanatory. 
# We consider a particle with unit mass
class VVIntegrator(object):
    """ Velocity-Verlet integrator """
    def __init__(self, force, dt, q):
        """ force is a function that takes a vector of positions q and returns -dV/dq """
        self.force = force
        self.dt = dt
        self.f = force(q)
    
    def step(self, q, p):        
        p[:] += self.f * self.dt *0.5
        q[:] += p * self.dt
        self.f = self.force(q)
        p[:] += self.f * self.dt*0.5            
        
class GLEIntegrator(object):
    """ Finite time-step GLE integrator for a free particle """
    def __init__(self, Ap, Cp, dt):
        self.ns = len(Ap)-1
        self.Ap = Ap
        self.Cp = Cp
        self.dt = dt
        self.T = sp.linalg.expm(-Ap*self.dt*0.5)
        self.S = sp.linalg.cholesky(self.Cp - self.T @ self.Cp @self.T.T).T
    
    def step(self, p, s):        
        ps = np.vstack([p,s])
        # stores the "GLE force" contribution for analysis
        self._rf = self.T @ ps - ps + self.S @ np.random.normal(size=(self.ns+1, len(p)))
        ps += self._rf        
        p[:] = ps[0]
        s[:] = ps[1:]

We can then run a trajectory for a free particle, using an exponential memory kernel. We run a 2D trajectory, but the two directions are equivalent

In [ ]:
# initialize the trajectory
q = np.asarray([0.,0.])
p = np.asarray([0.,0.])
s = np.asarray([[0.,0.]])
dt = 0.1
Ap = Ap_exp(0.2, 10, 0.01)  # default value: Ap_exp(0.2, 10, 0.01)
GLE = GLEIntegrator(Ap, np.eye(s.shape[0]+1), dt)
VV = VVIntegrator(lambda x:0.*x, dt, q)

In [ ]:
nstep = 40000 # default value: 40000
traj_q = np.zeros((nstep, len(q)))
traj_p = np.zeros((nstep, len(p)))
traj_f = np.zeros((nstep, len(p)))
for istep in range(nstep):
    traj_q[istep] = q; traj_p[istep] = p
    GLE.step(p,s)
    traj_f[istep] = GLE._rf[0]/(0.5*dt) # this is \dot{p} + V'(q), see the exercise in the lecture notes
    VV.step(q,p)
    GLE.step(p,s)    

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4), constrained_layout=True)
ax.plot(traj_q[:,0], traj_q[:,1], 'r-')
ax.set_xlabel("$q_1$"); ax.set_ylabel("$q_2$");

The trajectory of the particle is quite different from what you would get with a white-noise Langevin run. Experiment with different parameters and types of the `Ap` matrix. 
<p style="color:blue; font-weight:bold">
Run a white-noise simulation with a_pp = 0.2 (you can achieve this by manually constructing an Ap matrix, or by setting gamma=0 in the exponential kernel). Observe the difference in the trajectory behavior: can you recognize this out of several trajectories generated with colored noise?
</p>

<a id="equilibrium"> </a>

# Equilibrium GLE sampling

This section requires use of i-PI, so make sure you have it installed and have familiarized yourself with how to run it in the [getting started](0-getting_started.ipynb) section. 

We will set up and run a molecular dynamics simulation of liquid water using a "smart-sampling" GLE thermostat, using the parameter generator from the [GLE4MD website](https://gle4md.org). Given that we will compare the sampling efficiency to that obtained from white-noise Langevin simulations, you may want to run the exercises from section 4 of the [sampling and MD notebook](1-md_sampling.ipynb). 

For reference, these are the values of the correlation times for $V$ and $K$ obtained from a long simulation with different white noise thermostat relaxation time $\tau=1/\gamma$


|  $\tau$ / fs   |    $\tau_V$ / ps  |   $\tau_K$ / ps   |
|----------------|-------------------|-------------------|
|     1          |        10         |      0.0009       |
|    100         |        0.8        |      0.04         |
|   10000        |        4.6        |      1.5          |

We also load some reference data that was generated with those trajectories, that contains the correlation functions of potential and kinetic energy at different Langevin $\tau$ (format: `[time{ps}, acf_V(tau=1fs),   acf_K(tau=1fs), acf_V(tau=100fs), acf_K(tau=100fs), acf_V(tau=10ps), acf_K(tau=10ps)]`)

In [ ]:
ref_data = np.loadtxt('5-gle/ref_langevin_acf.dat')

Now we can prepare the GLE trajectory. Make a copy of the template file

```
$ cd pimd-mooc/5-gle
$ cp template_gle.xml input.xml
```

and modify it to use a GLE thermostat. We will use a "smart-sampling" GLE, so set the prefix to `md-gle_smart`, to match the post-processing below.

Then, the important part: setting the GLE parameters. We will use the on-line generator on the [GLE4MD](https://gle4md.org) website. The website does not fit parameters from scratch, but uses a library of pre-optimized parameters, and uses scaling rules to adjust them for the range of frequencies of interest. 

![the input generation interface on gle4md.org](figures/gle4md-screenshot.png)

"Smart" GLE thermostats are designed to provide optimal sampling efficiency for a characteristic time scale. We set it to 5 ps, given that the example is limited to 20ps and so 5-10ps is the longest time scale we can hope to target. By choosing a parameters preset that targets 3 orders of magnitude in frequency, we get "as efficient as possible" sampling up to about 6000 cm<sup>-1</sup>, well above the highest vibrational frequencies in water that are around 3600 cm<sup>-1</sup>. We set the formatting to i-PI output, that generates an XML block that should be copied and pasted within the `<dynamics>` block in the input file. 
You can get parameters that are suitable for water following [this link](https://gle4md.org/index.html?page=matrix&kind=smart&tslow=5&utslow=ps&smrange=6-3&outmode=ipi)

Having set up the input file, you can run it as usual, launching i-PI first, and then the driver code that computes q-TIP4P/f energies and forces. 

```
$ i-pi input.xml &> log &
$ i-pi-driver -u -h driver -m qtip4pf 
```

Wait until the run has completed, then load the output trajectory and continue the analysis

In [ ]:
traj_gle = pimdmooc.read_ipi_output('5-gle/md-gle_smart.out')

In [ ]:
traj_pos = pimdmooc.read_ipi_xyz('5-gle/md-gle_smart.pos_0.xyz')

We get the radial distribution functions, to get an idea of the structure of the liquid. These will also be used further down to compare with quantum simulations.
_NB: ASE normalizes partial RDF in such a way they do not tend to 1 for a homogeneous system. We correct manually the normalization_

In [ ]:
rbins = get_rdf(traj_pos[0], rmax=4.5, nbins=200, elements=[8, 8])[1] 
rdf_cls_oo = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 8])[0] for f in traj_pos[::10]]).mean(axis=0)/(1/3)
rdf_cls_oh = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 1])[0] for f in traj_pos[::10]]).mean(axis=0)/(2/3)
rdf_cls_hh = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[1, 1])[0] for f in traj_pos[::10]]).mean(axis=0)/(2/3)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(rbins, rdf_cls_oo, 'r-' )
ax.plot(rbins, rdf_cls_oh, c='gray' )
ax.plot(rbins, rdf_cls_hh, 'c-' )
ax.set_xlabel(r"$r / \AA$"); ax.set_ylabel(r"RDF"); 
ax.set_ylim(-0.1,5)

We compute the autocorrelation function of potential and kinetic energy for the trajectory. 

In [ ]:
acf_v_gle = pimdmooc.autocorrelate(traj_gle["potential"], normalize=True)
acf_k_gle = pimdmooc.autocorrelate(traj_gle["kinetic_md"], normalize=True)

In [ ]:
# integral-by-sum (we truncate at ~5ps because of the high level of noise)
tau_v = (acf_v_gle[:5000].sum() - 0.5*acf_v_gle[0])*traj_gle["time"][1]
tau_k = (acf_k_gle[:5000].sum() - 0.5*acf_k_gle[0])*traj_gle["time"][1]

In [ ]:
print("Autocorrelation time: tau_V = % 10.5f ps,  tau_K = % 10.5f ps" % (tau_v, tau_k))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3.5))
acf_len = 10000
ax[0].plot(ref_data[:acf_len,0], ref_data[:acf_len,2], color=(0.5,0,0,0.5), label=r"$K, \tau=1$ fs")
ax[0].plot(ref_data[:acf_len,0], ref_data[:acf_len,4], color=(1,0,0,0.5), label=r"$K, \tau=100$ fs")
ax[0].plot(ref_data[:acf_len,0], ref_data[:acf_len,6], color=(1,0.5,0.5,0.5), label=r"$K, \tau=10$ ps")
ax[0].plot(ref_data[:acf_len,0], acf_k_gle[:acf_len], color='k', label=r"$K, $ GLE")
ax[1].plot(ref_data[:acf_len,0], ref_data[:acf_len,1], color=(0,0,0.5,0.5), label=r"$V, \tau=1$ fs")
ax[1].plot(ref_data[:acf_len,0], ref_data[:acf_len,3], color=(0,0,1,0.5), label=r"$V, \tau=100$ fs")
ax[1].plot(ref_data[:acf_len,0], ref_data[:acf_len,5], color=(0.5,0.5,1,0.5), label=r"$V, \tau=10$ ps")
ax[1].plot(ref_data[:acf_len,0], acf_v_gle[:acf_len], color='k', label=r"$V, $ GLE")
for a in ax:
    a.legend(ncol=1)
    a.legend(ncol=1)
    a.set_xlabel("time / ps"); a.set_ylabel("energy / a.u.");

<p style="color:blue; font-weight:bold">
Observe the autocorrelation functions, and compare them with those obtained by white-noise thermostats. 
</p>
<em>NB: the reference trajectories are obtained from much longer simulations, so they have smaller error in the asymptotic regime. Those from the GLE run will be noisier so you have to use a bit of "mental filtering". You can set up the GLE run to be longer, if you can afford the wait!</em>

<p style="color:blue; font-weight:bold">
Imagine what would happen if you set the masses of all the atoms to be 100 times larger. What would you change in the setup of the simulation, what would change in the results and what will not?
</p>

<a id="non-equilibrium"> </a>

# Non-equilibrium GLE sampling